<a href="https://colab.research.google.com/github/dmsroysillerico/MAE_M13_PUB/blob/main/Sprint2B_Churn_Olist_DRIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Sprint2B – Churn de Clientes Olist (Sprint 1 + Sprint 2)

**Notebook:** `Sprint2B_Churn_Olist_DRIVE.ipynb`  
**Proyecto:** Análisis de *churn* de clientes (clientes que dejan de comprar) – Dataset Olist  
**Sprints cubiertos:**  
- **Sprint 1:** EDA inicial, hipótesis, definición preliminar de target y métricas base.  
- **Sprint 2:** Pipeline reproducible, feature engineering avanzado, definición de target final, modelos base y métricas de negocio.

> Este notebook está pensado como una versión **end‑to‑end** del proyecto.  
> El notebook original del Sprint 1 se mantiene separado como evidencia histórica.



## 0. Agenda del Notebook

1. **Contexto de negocio y objetivo del MVP**  
2. **Configuración del entorno y rutas**  
3. **Carga e inspección de datos Olist (9 CSV)**  
4. **EDA inicial resumido (Sprint 1)**  
   - RFM básico  
   - Primeras métricas de churn  
5. **Diseño del pipeline reproducible (Sprint 2)**  
   - Funciones modulares: ingesta, limpieza, master table, features, target  
6. **Construcción de Master Table a nivel pedido (`orders_enriched`)**  
7. **Agregación por cliente y Feature Engineering ampliado (`customer_features`)**  
   - Variables RFM  
   - Variables logísticas, de reviews, pagos, diversidad de compras, etc.  
8. **Definición de target de churn**  
   - Target preliminar (Sprint 1)  
   - Target final (Sprint 2)  
9. **Selección de variables (modelo para “redimensionar” – Random Forest)**  
10. **Modelos de clasificación (al menos dos modelos)**  
    - Regresión logística  
    - Random Forest  
11. **Métricas técnicas y de negocio**  
    - Accuracy, Precision, Recall, F1, ROC‑AUC  
    - Churn rate, retención, análisis por segmentos  
    - Ejemplos de correlación, chi‑cuadrado, ANOVA/F‑test  
12. **Notas sobre versionamiento (Git) y pipeline mensual**  



## 1. Contexto de negocio y objetivo del MVP

- **Problema de negocio:** Olist desea **identificar clientes con riesgo de abandono (churn)** para poder priorizar campañas de retención y evitar la pérdida de valor futuro.
- **Unidad de análisis:** cliente final, identificado por `customer_unique_id`.
- **Churn (concepto):** cliente que **compró en el pasado pero deja de comprar** por un período suficientemente largo como para considerarlo *cliente perdido* desde la perspectiva del negocio.
- **Periodicidad objetivo:** análisis **mensual** (clientes activos vs inactivos dentro de ventanas temporales).

### Objetivo del MVP

El **MVP** del proyecto es un pipeline que, **cada mes**, sea capaz de:

1. Leer los datos transaccionales de Olist (nuevos pedidos).  
2. Construir automáticamente una **tabla maestra a nivel cliente** con variables de comportamiento (RFM, logística, satisfacción, etc.).  
3. Generar una **probabilidad de churn** por cliente.  
4. Entregar una **lista priorizada** de clientes en alto riesgo de abandono, junto con **KPIs de negocio** (churn rate, retención, valor monetario esperado, etc.).

Este notebook implementa la versión **analítica y de prototipo** de ese MVP, cubriendo los entregables de Sprint 1 y Sprint 2.


## 2. Configuración del entorno e imports

In [ ]:

# Imports principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2, f_classif
from sklearn.metrics import ConfusionMatrixDisplay

warnings.filterwarnings("ignore")
plt.style.use("default")
pd.options.display.float_format = "{:,.4f}".format

print("Entorno listo.")



### 2.1. Configuración de rutas

Ajustar la ruta base `DATA_DIR` según el entorno:

- En **Google Drive / Colab**, por ejemplo:  
  `DATA_DIR = Path('/content/drive/MyDrive/Maestria/M13/Olist')`
- En entorno local, apuntar a la carpeta donde están los 9 CSV de Olist.

En este notebook los archivos esperados son:

- `olist_orders_dataset.csv`  
- `olist_customers_dataset.csv`  
- `olist_order_items_dataset.csv`  
- `olist_order_payments_dataset.csv`  
- `olist_order_reviews_dataset.csv`  
- `olist_products_dataset.csv`  
- `olist_sellers_dataset.csv`  
- `olist_geolocation_dataset.csv`  
- `product_category_name_translation.csv`


In [ ]:

# Ruta base configurable (MODIFICAR AQUÍ SEGÚN EL ENTORNO)
DATA_DIR = Path('/content/drive/MyDrive/Maestria/M13/Olist')

csv_files = [
    'olist_orders_dataset.csv',
    'olist_customers_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv',
    'olist_geolocation_dataset.csv',
    'product_category_name_translation.csv'
]

print("Archivos esperados:")
for f in csv_files:
    print(" -", DATA_DIR / f)


## 3. Carga de datos Olist

In [ ]:

def load_olist_data(data_dir: Path):
    """Carga los 9 CSV de Olist y aplica parsers básicos de fechas.

    Parameters
    ----------
    data_dir : Path
        Carpeta donde se encuentran los archivos CSV.

    Returns
    -------
    dict
        Diccionario con cada DataFrame.
    """
    orders = pd.read_csv(
        data_dir / 'olist_orders_dataset.csv',
        parse_dates=[
            'order_purchase_timestamp',
            'order_approved_at',
            'order_delivered_carrier_date',
            'order_delivered_customer_date',
            'order_estimated_delivery_date'
        ]
    )

    customers = pd.read_csv(data_dir / 'olist_customers_dataset.csv')

    order_items = pd.read_csv(
        data_dir / 'olist_order_items_dataset.csv',
        parse_dates=['shipping_limit_date']
    )

    order_payments = pd.read_csv(data_dir / 'olist_order_payments_dataset.csv')

    order_reviews = pd.read_csv(
        data_dir / 'olist_order_reviews_dataset.csv',
        parse_dates=['review_creation_date', 'review_answer_timestamp']
    )

    products = pd.read_csv(data_dir / 'olist_products_dataset.csv')
    sellers = pd.read_csv(data_dir / 'olist_sellers_dataset.csv')
    geolocation = pd.read_csv(data_dir / 'olist_geolocation_dataset.csv')
    prod_cat_trans = pd.read_csv(data_dir / 'product_category_name_translation.csv')

    data = {
        'orders': orders,
        'customers': customers,
        'order_items': order_items,
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'products': products,
        'sellers': sellers,
        'geolocation': geolocation,
        'prod_cat_trans': prod_cat_trans
    }
    return data

# Cargar datos
data = load_olist_data(DATA_DIR)

for name, df in data.items():
    print(f"{name:15s}: {df.shape[0]:7d} filas x {df.shape[1]:2d} columnas")



## 4. EDA inicial resumido (Sprint 1)

En el Sprint 1 se realizó un EDA más detallado en otro notebook.  
Aquí se resume lo esencial necesario para continuar el pipeline:

- **Rango temporal de las órdenes.**  
- **Número de clientes y pedidos.**  
- **Variables RFM básicas a nivel cliente.**


In [ ]:

orders = data['orders']
customers = data['customers']

print("Rango temporal de `order_purchase_timestamp`:")
print("  Mínimo:", orders['order_purchase_timestamp'].min())
print("  Máximo:", orders['order_purchase_timestamp'].max())
print("  Duración (días):",
      (orders['order_purchase_timestamp'].max() - orders['order_purchase_timestamp'].min()).days)

print("\nNúmero de pedidos y clientes:")
print("  Pedidos totales:", len(orders))
print("  Clientes (customer_id) únicos:", orders['customer_id'].nunique())
print("  Clientes (customer_unique_id) únicos:", customers['customer_unique_id'].nunique())


In [ ]:

def missing_values_table(df: pd.DataFrame, max_rows: int = 20):
    """Resumen de valores faltantes por columna."""
    mv = df.isna().sum()
    mv = mv[mv > 0].sort_values(ascending=False)
    res = pd.DataFrame({
        'missing_count': mv,
        'missing_pct': mv / len(df) * 100
    })
    return res.head(max_rows)

print("Valores faltantes en orders:")
display(missing_values_table(orders))



## 5. Diseño del pipeline reproducible (Sprint 2)

A partir de aquí se define un **pipeline modular** que será la base del MVP:

1. **Ingesta:** lectura de los CSV (ya implementado en `load_olist_data`).  
2. **Data Cleaning:** tratamiento de valores faltantes, tipos y filtros básicos.  
3. **Master Table a nivel pedido:** `orders_enriched`.  
4. **Agregación y Feature Engineering a nivel cliente:** `customer_features`.  
5. **Definición de target (preliminar y final).**  
6. **Preparación de datos para modelado.**  
7. **Entrenamiento y evaluación de modelos.**

El objetivo es que este pipeline pueda ejecutarse **de extremo a extremo** (end‑to‑end)
y, en el futuro, adaptarse a una ingesta mensual de nuevos datos.


### 5.1. Funciones de limpieza básica

In [ ]:

def clean_orders(df: pd.DataFrame) -> pd.DataFrame:
    """Limpieza básica de la tabla de órdenes.

    - Elimina filas sin `order_purchase_timestamp`.
    - Filtra estados desconocidos si es necesario.
    """
    df = df.copy()
    df = df[~df['order_purchase_timestamp'].isna()]
    return df

def clean_customers(df: pd.DataFrame) -> pd.DataFrame:
    """Limpieza básica de clientes.

    (Aquí se podría:
     - homogeneizar ciudades/estados,
     - tratar zip codes,
     - eliminar duplicados, etc.)
    """
    df = df.copy()
    df = df.drop_duplicates(subset=['customer_id'])
    return df

def clean_order_payments(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    return df

def clean_order_reviews(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    return df

def clean_order_items(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    return df

# Aplicar limpieza
orders_clean = clean_orders(data['orders'])
customers_clean = clean_customers(data['customers'])
order_items_clean = clean_order_items(data['order_items'])
order_payments_clean = clean_order_payments(data['order_payments'])
order_reviews_clean = clean_order_reviews(data['order_reviews'])

print("Limpieza básica aplicada.")


## 6. Master table a nivel pedido: `orders_enriched`

In [ ]:

def build_orders_enriched(
    orders: pd.DataFrame,
    customers: pd.DataFrame,
    order_items: pd.DataFrame,
    order_payments: pd.DataFrame,
    order_reviews: pd.DataFrame
) -> pd.DataFrame:
    """Construye una tabla maestra a nivel pedido.

    Pasos:
    1. Orders + Customers.
    2. Agregación de pagos por `order_id`.
    3. Agregación de reviews por `order_id`.
    4. Agregación de items por `order_id` (precio, flete, nº ítems, nº sellers).
    5. Cálculo de tiempos logísticos (demoras).
    6. Filtro a órdenes entregadas (`delivered`).
    """
    # 1) Orders + Customers
    df = orders.merge(customers, on='customer_id', how='left')

    # 2) Pagos agregados
    payments_agg = order_payments.groupby('order_id').agg(
        total_payment=('payment_value', 'sum'),
        avg_installments=('payment_installments', 'mean')
    ).reset_index()

    df = df.merge(payments_agg, on='order_id', how='left')

    # 3) Reviews agregadas
    reviews_agg = order_reviews.groupby('order_id').agg(
        avg_review_score=('review_score', 'mean'),
        n_reviews=('review_score', 'size')
    ).reset_index()

    df = df.merge(reviews_agg, on='order_id', how='left')

    # 4) Items agregados
    items_agg = order_items.groupby('order_id').agg(
        n_items=('order_item_id', 'count'),
        n_sellers=('seller_id', 'nunique'),
        total_items_price=('price', 'sum'),
        total_freight_value=('freight_value', 'sum')
    ).reset_index()

    items_agg['order_value'] = items_agg['total_items_price'] + items_agg['total_freight_value']
    df = df.merge(items_agg, on='order_id', how='left')

    # 5) Tiempos logísticos
    df['purchase_to_approval_days'] = (
        (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 86400
    )
    df['approval_to_carrier_days'] = (
        (df['order_delivered_carrier_date'] - df['order_approved_at']).dt.total_seconds() / 86400
    )
    df['carrier_to_customer_days'] = (
        (df['order_delivered_customer_date'] - df['order_delivered_carrier_date']).dt.total_seconds() / 86400
    )
    df['delivery_delay_days'] = (
        (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.total_seconds() / 86400
    )

    # 6) Quedarnos solo con órdenes entregadas
    df_completed = df[df['order_status'] == 'delivered'].copy()

    return df_completed

orders_enriched = build_orders_enriched(
    orders_clean,
    customers_clean,
    order_items_clean,
    order_payments_clean,
    order_reviews_clean
)

print("orders_enriched:", orders_enriched.shape)
display(orders_enriched.head())


## 7. Agregación por cliente y Feature Engineering ampliado

In [ ]:

def build_customer_features(orders_enriched: pd.DataFrame) -> pd.DataFrame:
    """Agrega información a nivel `customer_unique_id` y crea features de comportamiento.

    Incluye:
    - Variables RFM.
    - Variables logísticas.
    - Variables monetarias.
    - Diversidad de compras (sellers, items).
    - Indicadores de reviews.
    """
    df = orders_enriched.copy()

    # Aseguramos existencia de columnas clave
    max_date = df['order_purchase_timestamp'].max()

    agg_dict = {
        'order_purchase_timestamp': ['min', 'max', 'count'],
        'order_value': ['sum', 'mean', 'std', 'median', 'max'],
        'total_items_price': ['sum', 'mean', 'std'],
        'total_freight_value': ['sum', 'mean', 'std'],
        'n_items': ['sum', 'mean', 'max'],
        'n_sellers': ['sum', 'mean', 'max'],
        'total_payment': ['sum', 'mean'],
        'avg_installments': ['mean'],
        'avg_review_score': ['mean', 'std'],
        'n_reviews': ['sum'],
        'purchase_to_approval_days': ['mean', 'std'],
        'approval_to_carrier_days': ['mean', 'std'],
        'carrier_to_customer_days': ['mean', 'std'],
        'delivery_delay_days': ['mean', 'std', lambda x: (x > 0).sum(), lambda x: (x < 0).sum()]
    }

    customer_features = (
        df.groupby('customer_unique_id')
          .agg(agg_dict)
          .reset_index()
    )

    # Renombrar columnas multi-índice
    customer_features.columns = [
        '_'.join(col).strip('_') if isinstance(col, tuple) else col
        for col in customer_features.columns.values
    ]

    # Renombrados más amigables para algunas columnas clave
    rename_map = {
        'order_purchase_timestamp_min': 'first_purchase_date',
        'order_purchase_timestamp_max': 'last_purchase_date',
        'order_purchase_timestamp_count': 'frequency',
        'order_value_sum': 'monetary_total',
        'order_value_mean': 'monetary_avg',
        'order_value_std': 'monetary_std',
        'order_value_median': 'monetary_median',
        'order_value_max': 'monetary_max',
        'total_items_price_sum': 'items_price_total',
        'total_freight_value_sum': 'freight_total',
        'delivery_delay_days_mean': 'avg_delivery_delay',
        'delivery_delay_days_<lambda_0>': 'num_delayed_orders',
        'delivery_delay_days_<lambda_1>': 'num_early_orders',
        'avg_review_score_mean': 'avg_review_score',
        'avg_review_score_std': 'std_review_score',
        'n_reviews_sum': 'n_reviews'
    }
    customer_features = customer_features.rename(columns=rename_map)

    # Variables RFM & derivadas
    customer_features['recency_days'] = (
        (max_date - customer_features['last_purchase_date']).dt.days
    )
    customer_features['customer_lifetime_days'] = (
        (customer_features['last_purchase_date'] - customer_features['first_purchase_date']).dt.days
    )
    customer_features['avg_days_between_orders'] = (
        customer_features['customer_lifetime_days'] / (customer_features['frequency'] - 1)
    )
    customer_features['avg_days_between_orders'] = customer_features['avg_days_between_orders'].replace(
        [np.inf, -np.inf], np.nan
    ).fillna(0)

    # Porcentaje de órdenes atrasadas y adelantadas
    customer_features['pct_delayed_orders'] = (
        customer_features['num_delayed_orders'] / customer_features['frequency']
    ).fillna(0)
    customer_features['pct_early_orders'] = (
        customer_features['num_early_orders'] / customer_features['frequency']
    ).fillna(0)

    # Join con estado del cliente (primer estado observado)
    state_map = (
        df.groupby('customer_unique_id')['customer_state']
          .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
    )
    customer_features = customer_features.merge(
        state_map.reset_index(),
        on='customer_unique_id',
        how='left'
    )

    return customer_features

customer_features = build_customer_features(orders_enriched)
print("customer_features:", customer_features.shape)
display(customer_features.head())


## 8. Definición de target de churn (preliminar y final)

In [ ]:

# Target preliminar (Sprint 1): recency > 90 días
PRELIM_CHURN_THRESHOLD = 90

customer_features['churn_prelim'] = (
    customer_features['recency_days'] > PRELIM_CHURN_THRESHOLD
).astype(int)

# Resumen de distribución preliminar
def print_churn_distribution(label_col: str):
    churn_count = customer_features[label_col].sum()
    total = len(customer_features)
    active_count = total - churn_count
    churn_rate = churn_count / total * 100
    print(f"Target: {label_col}")
    print(f"  Clientes totales : {total:6d}")
    print(f"  Churned (1)      : {churn_count:6d} ({churn_rate:5.1f}%)")
    print(f"  Activos  (0)     : {active_count:6d} ({100 - churn_rate:5.1f}%)\n")

print_churn_distribution('churn_prelim')

# Target final (Sprint 2) – EJEMPLO:
# Se usa un umbral más laxo para reducir el desbalance, por ejemplo 365 días.
FINAL_CHURN_THRESHOLD = 365

customer_features['churn_final'] = (
    customer_features['recency_days'] > FINAL_CHURN_THRESHOLD
).astype(int)

print_churn_distribution('churn_final')

# Seleccionaremos 'churn_final' como target para modelado
TARGET_COL = 'churn_final'


### 8.1. Correlaciones básicas con el target

In [ ]:

numeric_vars = [
    col for col in customer_features.select_dtypes(include=[np.number]).columns
    if col not in ['churn_prelim', 'churn_final']
]

corr_rows = []
for var in numeric_vars:
    corr = customer_features[[var, TARGET_COL]].corr().iloc[0, 1]
    corr_rows.append({'Variable': var, 'Correlation': corr, 'Abs': abs(corr)})

corr_df = (
    pd.DataFrame(corr_rows)
    .dropna()
    .sort_values('Abs', ascending=False)
)

print("Top 15 correlaciones (por |correlación|):\n")
for _, row in corr_df.head(15).iterrows():
    strength = (
        "FUERTE" if abs(row['Correlation']) > 0.5
        else "MODERADA" if abs(row['Correlation']) > 0.3
        else "DÉBIL"
    )
    print(f"  {row['Variable']:35s}: {row['Correlation']:+.3f} ({strength})")

plt.figure(figsize=(8, 6))
sns.barplot(
    data=corr_df.head(15),
    y='Variable', x='Correlation',
    palette=['red' if c > 0 else 'blue' for c in corr_df.head(15)['Correlation']]
)
plt.axvline(0, color='black', linewidth=0.8)
plt.title('Top correlaciones con el target')
plt.tight_layout()
plt.show()


## 9. Preparación de datos para modelado

In [ ]:

# Seleccionamos variables numéricas y categóricas simples
feature_cols_numeric = [
    col for col in customer_features.select_dtypes(include=[np.number]).columns
    if col not in ['churn_prelim', 'churn_final']
]
feature_cols_categ = ['customer_state']

X_num = customer_features[feature_cols_numeric].fillna(0)
X_cat = pd.get_dummies(customer_features[feature_cols_categ], drop_first=True)

X = pd.concat([X_num, X_cat], axis=1)
y = customer_features[TARGET_COL]

print("Shape de X, y:", X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Train / Test listos para modelado.")


## 10. Modelos de clasificación

### 10.1. Modelo base: Regresión Logística

In [ ]:

log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'  # ayuda en caso de desbalance residual
)
log_reg.fit(X_train_scaled, y_train)

y_pred_lr = log_reg.predict(X_test_scaled)
y_proba_lr = log_reg.predict_proba(X_test_scaled)[:, 1]

print("Reporte de clasificación – Regresión Logística")
print(classification_report(y_test, y_pred_lr, digits=3))

print("ROC-AUC:", roc_auc_score(y_test, y_proba_lr))

ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred_lr,
    display_labels=['Activo', 'Churn'],
    cmap='Blues'
)
plt.title('Matriz de confusión – Regresión Logística')
plt.show()


### 10.2. Modelo 2: Random Forest (y selección de variables)

In [ ]:

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    class_weight='balanced'
)
rf.fit(X_train, y_train)  # Random Forest maneja bien escalas distintas

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("Reporte de clasificación – Random Forest")
print(classification_report(y_test, y_pred_rf, digits=3))

print("ROC-AUC:", roc_auc_score(y_test, y_proba_rf))

ConfusionMatrixDisplay.from_predictions(
    y_test, y_pred_rf,
    display_labels=['Activo', 'Churn'],
    cmap='Greens'
)
plt.title('Matriz de confusión – Random Forest')
plt.show()

# Importancias de variables (para redimensionar / seleccionar)
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)

print("\nTop 20 features más importantes según Random Forest:")
display(importances.head(20).to_frame('importance'))

plt.figure(figsize=(8, 6))
sns.barplot(x=importances.head(20), y=importances.head(20).index)
plt.title('Importancia de las 20 principales variables')
plt.tight_layout()
plt.show()


## 11. Métricas de negocio y pruebas estadísticas (Chi², ANOVA/F‑test)

In [ ]:

# Ejemplo sencillo de pruebas estadísticas sobre un subconjunto de variables

# Para Chi² se requieren variables no negativas
from sklearn.preprocessing import MinMaxScaler
scaler_mm = MinMaxScaler()
X_chi2 = scaler_mm.fit_transform(X_num)

chi2_vals, chi2_p = chi2(X_chi2, y)
chi2_results = pd.DataFrame({
    'feature': feature_cols_numeric,
    'chi2_stat': chi2_vals,
    'p_value': chi2_p
}).sort_values('p_value')

print("Top 10 variables (Chi², menor p-value):")
display(chi2_results.head(10))

# ANOVA / F-test (para relación lineal con el target)
f_vals, f_p = f_classif(X_num, y)
f_results = pd.DataFrame({
    'feature': feature_cols_numeric,
    'f_stat': f_vals,
    'p_value': f_p
}).sort_values('p_value')

print("\nTop 10 variables (ANOVA F-test, menor p-value):")
display(f_results.head(10))


### 11.1. KPIs de churn y retención

In [ ]:

def compute_basic_kpis(df_cf: pd.DataFrame, target_col: str):
    churn_rate = df_cf[target_col].mean()
    retention_rate = 1 - churn_rate
    avg_monetary = df_cf['monetary_total'].mean()
    active_monetary = df_cf.loc[df_cf[target_col] == 0, 'monetary_total'].mean()
    churned_monetary = df_cf.loc[df_cf[target_col] == 1, 'monetary_total'].mean()

    print(f"Churn rate        : {churn_rate*100:5.2f}%")
    print(f"Retention rate    : {retention_rate*100:5.2f}%")
    print(f"Gasto promedio total por cliente : {avg_monetary:,.2f}")
    print(f"Gasto promedio clientes activos  : {active_monetary:,.2f}")
    print(f"Gasto promedio clientes churn    : {churned_monetary:,.2f}")

compute_basic_kpis(customer_features, TARGET_COL)



## 12. Notas sobre versionamiento (Git) y pipeline mensual

- Este notebook debe estar versionado en un repositorio Git, junto con:
  - El notebook original del **Sprint 1**.
  - Un README explicando:
    - Objetivo del proyecto.
    - Estructura de carpetas.
    - Cómo ejecutar el pipeline de principio a fin.
- Sugerencia de pasos mínimos en Git:
  1. `git init` en la carpeta del proyecto.  
  2. Añadir `.gitignore` (por ejemplo, para datos crudos si no se suben).  
  3. `git add .`  
  4. `git commit -m "Sprint 1 y 2 - pipeline churn Olist"`  

### Simulación de ejecución mensual

La idea para futuras iteraciones es:

1. Cada mes se agregan nuevas filas a `olist_orders_dataset.csv` y tablas relacionadas.  
2. El pipeline de este notebook se encapsula en funciones (por ejemplo `run_monthly_pipeline(fecha_corte)`),
   que vuelven a:
   - cargar datos,  
   - construir `orders_enriched`,  
   - recalcular `customer_features`,  
   - marcar `churn_final` según la ventana de recencia definida,  
   - generar un ranking actualizado de clientes en riesgo.

De esta forma, el MVP queda listo para integrarse posteriormente a un dashboard o a un sistema operativo de campañas de retención.
